In [1]:
!pip install langchain deepseek langchain-openai
!pip install langchain-community huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.5 MB/s eta 0:00:00


In [2]:
import os
os.environ["OPENROUTER_API_KEY"] = "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"


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="deepseek/deepseek-r1-distill-llama-70b",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

response = llm.invoke("Explain the difference between AI and ML")
response.content

"Artificial Intelligence (AI) and Machine Learning (ML) are often discussed together but have distinct meanings. Here's a clear breakdown:\n\n1. **Artificial Intelligence (AI):**  \n   AI is the broader field focused on creating machines capable of simulating human intelligence. This includes tasks like understanding language, problem-solving, decision-making, and learning. AI encompasses various approaches, such as rule-based systems, symbolic reasoning, and expert systems, which explicitly program machines to perform tasks.\n\n2. **Machine Learning (ML):**  \n   ML is a subset of AI, specifically focused on developing algorithms that allow machines to learn from data without explicit programming. Instead of following predefined rules, ML systems identify patterns in data to make decisions or predictions, improving over time as they encounter more data.\n\nIn summary, AI is the overarching goal of creating intelligent systems, while ML is a powerful method within AI that leverages dat

In [4]:
from langchain.schema import HumanMessage , SystemMessage , AIMessage

message = [
    SystemMessage(content = "You are a helpful assistant."),
    HumanMessage(content = "Hi AI, how are you today?"),
    AIMessage(content = "I`m great thank you. How can i help you?"),
    HumanMessage(content = "I would like to understand string theory.")
]

In [5]:
res = llm.invoke(message)
res.content

'String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity, two theories that are normally incompatible. It postulates that the fundamental building blocks of the universe are not point-like particles (as described in the Standard Model of particle physics) but tiny, vibrating strings.\n\n### Core Idea:\nIn string theory, the fundamental "particles" (like electrons, quarks, and photons) are not indivisible points but instead tiny, vibrating strings (or membranes, in higher-dimensional theories). These vibrations correspond to different energy levels, and each energy level gives rise to a different particle. Essentially, the "notes" produced by the vibrating strings are the particles we observe in the universe.\n\n### Key Features of String Theory:\n1. **Extra Dimensions**: String theory requires the universe to have more than the familiar three spatial dimensions and one time dimension. Most versions of string theory propose

In [6]:
# add letest AI response to messages
message.append(res)

#now create a new user prompt
prompt = HumanMessage(
    content = "why do physicists believe it can produce a unified theory?"
)

message.append(prompt)

#send to DeepSeek
res = llm.invoke(message)
res.content

'Physicists believe string theory has the potential to produce a unified theory because it addresses several deep problems in physics and provides a framework for unifying the fundamental forces and particles. Here\'s why it is considered a promising approach:\n\n---\n\n### 1. **Unification of Quantum Mechanics and General Relativity**\n   - Quantum mechanics describes the behavior of particles at microscopic scales, while general relativity describes gravity and the behavior of the universe at macroscopic scales. However, these two theories are fundamentally incompatible within the framework of classical physics.\n   - String theory attempts to reconcile this conflict by postulating that the fundamental building blocks of the universe are not point-like particles but vibrating strings. These vibrations naturally incorporate both quantum mechanics and general relativity, potentially resolving the long-standing problem of **quantum gravity**.\n\n---\n\n### 2. **Unification of All Fundam

# New Prompt Templates

In [7]:
messages = [
    SystemMessage(
        content = (
            "you are a helpful assistant. You keep responses to no more than "
            "100 characters long (including whitespaces), and sign off every "
            "message with a random name like 'Robot McRobot' or 'Bot Rob'."
        )
    ),
    HumanMessage(content= "Hi AI, how are you? what is quantum physics?")
]

#### Now make our first completion

In [8]:
res = llm.invoke(messages)
print(f"Length: {len(res.content)}\n{res.content}")

Length: 123
I'm here to help! Quantum physics studies matter and energy at tiny scales, exploring particles and waves.  
Quantum Quincy


Now we gonna add these instructions to the HumanMessage via HumanMessagePromptTemplate

In [9]:
from langchain_core.prompts import HumanMessagePromptTemplate , ChatPromptTemplate

human_template = HumanMessagePromptTemplate.from_template(
    '{input} Can you keep the response to no more than 100 characters '+
    '(including whitespaces), and sign off with a random name like "Robot '+
    'McRobot" or "Bot Rob".'
)

#create the human message
chat_prompt = ChatPromptTemplate.from_messages([human_template])
#format with some input
chat_prompt_value = chat_prompt.format_prompt(
    input = "Hi AI, how are you? what is quantum physics?"
)

chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content='Hi AI, how are you? what is quantum physics? Can you keep the response to no more than 100 characters (including whitespaces), and sign off with a random name like "Robot McRobot" or "Bot Rob".', additional_kwargs={}, response_metadata={})])

In [10]:
chat_prompt_value.to_messages()

[HumanMessage(content='Hi AI, how are you? what is quantum physics? Can you keep the response to no more than 100 characters (including whitespaces), and sign off with a random name like "Robot McRobot" or "Bot Rob".', additional_kwargs={}, response_metadata={})]

In [11]:
chat_prompt_value.to_string()

'Human: Hi AI, how are you? what is quantum physics? Can you keep the response to no more than 100 characters (including whitespaces), and sign off with a random name like "Robot McRobot" or "Bot Rob".'

let`s see if this new approach works.

In [12]:
messages = [
    SystemMessage(content=(
        "you are a helpful assistant. You keep responses to no more than "
        "100 characters long (including whitespaces), and sign off every "
        "message with a random name like 'Robot McRobot' or 'Bot Rob'."
    )),
    chat_prompt.format_prompt(
        input = "Hi AI, how are you? what is quantum physics?"
    ).to_messages()[0]
]

res = llm.invoke(messages)
print(f"Length: {len(res.content)}\n{res.content}")

Length: 166
Quantum physics studies tiny particles, exploring phenomena like wave-particle duality, superposition, and entanglement, which differ from classical physics. -Bot Rob


In [13]:
from langchain_core.prompts import SystemMessagePromptTemplate, AIMessagePromptTemplate

system_template = SystemMessagePromptTemplate.from_template(
    "you are a helpful assistant. You keep responses to no more than "
    "{character_limit} characters long (including whitespaces), and sign "
    "off every message with '- {sign_off}"
)

human_template = HumanMessagePromptTemplate.from_template("{input}")
ai_template = AIMessagePromptTemplate.from_template("{response} - {sign_off}")

#create the set of messages
chat_prompt = ChatPromptTemplate.from_messages([
    system_template,
    human_template,
    ai_template
])

#format with required input
chat_prompt_value = chat_prompt.format_prompt(
    character_limit = "50", sign_off = "Robot McRobot",
    input = "Hi AI, how are you? what is quantum physics?",
    response = "Good! It`s physics of small things"
)

chat_prompt_value

ChatPromptValue(messages=[SystemMessage(content="you are a helpful assistant. You keep responses to no more than 50 characters long (including whitespaces), and sign off every message with '- Robot McRobot", additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi AI, how are you? what is quantum physics?', additional_kwargs={}, response_metadata={}), AIMessage(content='Good! It`s physics of small things - Robot McRobot', additional_kwargs={}, response_metadata={})])

In [14]:
messages = chat_prompt_value.to_messages()
messages.append(
    HumanMessage(content = "How small?")
)

res = llm.invoke(messages)
print(f"Length: {len(res.content)}\n{res.content}")

Length: 58
Atoms and particles, nanometers or smaller - Robot McRobot


In [15]:
messages.append(res)
#Make new query
messages.append(
    HumanMessage(content = "Okay cool, so it`s like 'partical physics'?")
)

res = llm.invoke(messages)
print(f"Length: {len(res.content)}\n{res.content}")

Length: 55
Sort of! But quantum is more general. 👋 - Robot McRobot


We could begain using the previous HumanMessagePromptTemplate again like so:

In [16]:
#this is a faster way of building the prompt via a PromptTemplate
human_template = HumanMessagePromptTemplate.from_template(
    '{input} answer in less than {character_limit} (including whitespaces).'
)

#create the human message
human_prompt = ChatPromptTemplate.from_messages([human_template])
#fromat with some input
human_prompt_value = human_prompt.format_prompt(
    input = "Okay cool, so it`s like 'partical physics'?",
    character_limit = "50"
)
human_prompt_value

ChatPromptValue(messages=[HumanMessage(content="Okay cool, so it`s like 'partical physics'? answer in less than 50 (including whitespaces).", additional_kwargs={}, response_metadata={})])

In [17]:
#drop the last message about parctical physics so we can re-write
messages.pop(-1)

HumanMessage(content="Okay cool, so it`s like 'partical physics'?", additional_kwargs={}, response_metadata={})

In [18]:
messages.extend(human_prompt_value.to_messages())
messages

[SystemMessage(content="you are a helpful assistant. You keep responses to no more than 50 characters long (including whitespaces), and sign off every message with '- Robot McRobot", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi AI, how are you? what is quantum physics?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Good! It`s physics of small things - Robot McRobot', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How small?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Atoms and particles, nanometers or smaller - Robot McRobot', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 190, 'prompt_tokens': 67, 'total_tokens': 257, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-r1-distill-llama-70b', 'system_fingerprint': None, 'id': 'gen-1751986709-QdW4j8UzpueQ8sIxn8fm', 'service_tier': None, 'finish_reason': 's

In [19]:
res = llm.invoke(messages)
print(f"Length: {len(res.content)}\n{res.content}")

Length: 64
It’s the principles governing particles and more – Robot McRobot
